AWS versus ERA Interim
======

Import custom modules for working with model and AWS data
----

In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
from IPython.display import HTML

from model import Model
from GCNet import GCNet

Read/create ERA Interim metadata
----

In [2]:
# M = Model("erai_geog.nc")
M = Model("erai_geog_sub.nc")
M.loadMeta()
# M.info()
# dir(M)
# M.getLat2d()
# M.createMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

1d dataset


Load AWS metadata
----

In [18]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

In [19]:
qcSub = A.loadMeta('qc_AirT1_1996-2005')
qcFull = A.loadMeta('qc_AirT1_1996-2014')
# print qcSub
# print qcFull

Find closest grid points to each AWS
-----

In [20]:
C = []
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    """ Basic AWS info """
    A.info(awsIX)
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    awsElev = A.getElev( awsIX )
    
    """ Find closest points """
    closest = M.closestPoints( awsLatLon )
    C.append( closest )
    
    """ Print info on closest points """
    print "Closest model points"
    P = []
    for d in closest:
        px = d['ixLon']
        py = d['ixLat']
        pdist = d['distance']
        plat  = M.getLat2d(py,px)
        plon  = M.getLon2d(py,px)
        pelev = M.getElev(py,px)
        perr = pelev - awsElev
        L = (py, px, pdist, plat, plon, pelev, perr)
        print "G(%d,%d): M: D = %.2f km, lat/lon/elev = %.2f/%.2f/%.2f m, elev_err = %.2f m" \
                     % L
        P.append( L ) 
    print "-"*80


Site 1: Swiss_Camp
Lat, Lon: 69.5732 310.7048
Elevation: 1176.0
Closest model points
G(23,29): M: D = 28.08 km, lat/lon/elev = 69.82/310.78/1057.57 m, elev_err = -118.43 m
G(23,28): M: D = 36.94 km, lat/lon/elev = 69.82/310.08/770.82 m, elev_err = -405.18 m
G(24,29): M: D = 50.22 km, lat/lon/elev = 69.12/310.78/909.66 m, elev_err = -266.34 m
G(24,28): M: D = 55.83 km, lat/lon/elev = 69.12/310.08/603.85 m, elev_err = -572.15 m
--------------------------------------------------------------------------------
Site 2: Crawford_Point1
Lat, Lon: 69.8819 313.0264
Elevation: 2022.0
Closest model points
G(23,32): M: D = 8.25 km, lat/lon/elev = 69.82/312.89/1845.64 m, elev_err = -176.36 m
G(23,33): M: D = 22.66 km, lat/lon/elev = 69.82/313.59/2020.26 m, elev_err = -1.74 m
G(22,32): M: D = 71.82 km, lat/lon/elev = 70.53/312.89/2043.40 m, elev_err = 21.40 m
G(22,33): M: D = 74.76 km, lat/lon/elev = 70.53/313.59/2197.32 m, elev_err = 175.32 m
---------------------------------------------------------

Closest model points
G(24,28): M: D = 33.52 km, lat/lon/elev = 69.12/310.08/603.85 m, elev_err = 35.85 m
G(24,27): M: D = 39.92 km, lat/lon/elev = 69.12/309.38/355.43 m, elev_err = -212.57 m
G(23,28): M: D = 45.27 km, lat/lon/elev = 69.82/310.08/770.82 m, elev_err = 202.82 m
G(23,27): M: D = 50.05 km, lat/lon/elev = 69.82/309.38/484.17 m, elev_err = -83.83 m
--------------------------------------------------------------------------------
Site 18: KULU
Lat, Lon: 65.7584 320.3982
Elevation: 878.0
Closest model points
G(29,43): M: D = 19.15 km, lat/lon/elev = 65.61/320.63/642.58 m, elev_err = -235.42 m
G(29,42): M: D = 27.10 km, lat/lon/elev = 65.61/319.92/958.89 m, elev_err = 80.89 m
G(28,43): M: D = 62.81 km, lat/lon/elev = 66.32/320.63/1461.44 m, elev_err = 583.44 m
G(28,42): M: D = 65.60 km, lat/lon/elev = 66.32/319.92/1691.82 m, elev_err = 813.82 m
--------------------------------------------------------------------------------
Site 19: JAR3
Lat, Lon: 69.3954 309.6896
Elevation: 323.

Interpolate model elevation to AWS location
====

In [21]:
df = pd.DataFrame(columns=['Site #','Site name','Site lat','Site lon','Site elev','ElevCorr','ElevErr',
                          'QC Full', 'QC Overlap','Dry T Adj','Moist T Adj'],index=range(nSites))
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    """ Basic AWS info """
    A.info(awsIX)
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    awsElev = A.getElev( awsIX )

    """ Perform the interpolation on elev from model metadata """
    closest = C[awsIX]
    Z = [ M.getElev( d[0], d[1] ) for d in zip( closest['ixLat'], closest['ixLon'] )]
    Znew = M.interpolate( Z, closest, awsLatLon )
    perr = Znew - awsElev
    print "\nNew elev = %0.2f, elev_err = %0.2f" % ( Znew, perr )
    dryAdAdj = perr/1000.*9.8
    mstAdAdj = perr/1000.*6.0
    row = ( awsIX+1, awsName, "%.2f" % awsLatLon[0], "%.2f" % awsLatLon[1], awsElev, \
            "%.2f" % Znew, "%.2f" % perr, \
            "%.2f" % qcFull[awsIX], "%.2f" % qcSub[awsIX], "%.2f" % dryAdAdj, "%.2f" % mstAdAdj )
    df.loc[awsIX] = row
    print "-"*80

Site 1: Swiss_Camp
Lat, Lon: 69.5732 310.7048
Elevation: 1176.0

New elev = 973.00, elev_err = -203.00
--------------------------------------------------------------------------------
Site 2: Crawford_Point1
Lat, Lon: 69.8819 313.0264
Elevation: 2022.0

New elev = 1895.42, elev_err = -126.58
--------------------------------------------------------------------------------
Site 3: NASA-U
Lat, Lon: 73.8333 310.5047
Elevation: 2334.0

New elev = 2319.32, elev_err = -14.68
--------------------------------------------------------------------------------
Site 4: GITS
Lat, Lon: 77.1433 298.905
Elevation: 1869.0

New elev = 1620.95, elev_err = -248.05
--------------------------------------------------------------------------------
Site 5: Humboldt
Lat, Lon: 78.5266 303.1695
Elevation: 1995.0

New elev = 1965.60, elev_err = -29.40
--------------------------------------------------------------------------------
Site 6: Summit
Lat, Lon: 72.5794 321.4958
Elevation: 3199.0

New elev = 3171.18, elev_

In [22]:
mstAdAdj

-0.12910065225166273

In [23]:
df

,Site #,Site name,Site lat,Site lon,Site elev,ElevCorr,ElevErr,QC Full,QC Overlap,Dry T Adj,Moist T Adj
0,1,Swiss Camp,69.57,310.70,1176,973.00,-203.00,69.25,75.07,-1.99,-1.22
1,2,Crawford Point1,69.88,313.03,2022,1895.42,-126.58,61.92,73.03,-1.24,-0.76
2,3,NASA-U,73.83,310.50,2334,2319.32,-14.68,51.73,35.75,-0.14,-0.09
3,4,GITS,77.14,298.90,1869,1620.95,-248.05,23.73,19.65,-2.43,-1.49
4,5,Humboldt,78.53,303.17,1995,1965.60,-29.40,55.30,65.07,-0.29,-0.18
5,6,Summit,72.58,321.50,3199,3171.18,-27.82,76.64,70.59,-0.27,-0.17
6,7,Tunu-N,78.02,326.01,2052,2077.42,25.42,80.45,81.17,0.25,0.15
7,8,DYE-2,66.48,313.72,2099,2100.99,1.99,87.17,93.48,0.02,0.01
8,9,JAR1,69.50,310.32,932,795.82,-136.18,77.37,78.08,-1.33,-0.82
9,10,Saddle,66.00,315.50,2467,2386.55,-80.45,77.54,86.97,-0.79,-0.48


In [24]:
# save the dataframe of AWS info
df.to_pickle('erai_site_info.pkl' )

In [25]:
# print a DataFrame to HTML file (then print that to PDF in a browser!)
hout = HTML(df.to_html())
hfile = open("aws_table.htm","w")
hfile.write(hout.data)
hfile.close()